# Home 4: Build a seq2seq model for machine translation.

### Name: [Ayush Yajnik]

### Task: Translate English to [Spanish]

## 0. You will do the following:

1. Read and run my code.
2. Complete the code in Section 1.1 and Section 4.2.

    * Translation English to **German** is not acceptable!!! Try another language.
    
3. **Make improvements.** Directly modify the code in Section 3. Do at least one of the followings. By doing more, you will get up to 2 bonus scores to the total.

    * Bi-LSTM instead of LSTM
    
    * Multi-task learning (e.g., both English to French and English to Spanish)
    
    * Attention
    
4. Evaluate the translation using the BLEU score. 

    * Optional. Up to 1 bonus scores to the total.
    
5. Convert the notebook to .HTML file. 

    * The HTML file must contain the code and the output after execution.

6. Put the .HTML file in your Google Drive, Dropbox, or Github repo. 

7. Submit the link to the HTML file to Canvas.    

### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder; the decoder won't be much different.

In [1]:
from keras.layers import Bidirectional, Concatenate
from keras import layers

encoder_bilstm = Bidirectional(layers.LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

Using TensorFlow backend.


NameError: ignored

In [0]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

In [0]:
# e.g., filename = 'Data/deu.txt'
filename = 'spa.txt'

# e.g., n_train = 20000
n_train = 25000

In [0]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[i loved that] => [eso me gusto]
[i loved that] => [eso me gustaba]
[i loved that] => [a mi me gustaba eso]
[i made a bet] => [hice una apuesta]
[i made these] => [yo hice estos]
[i made these] => [estos los hice yo]
[i misled tom] => [yo engane a tom]
[i missed you] => [te eche de menos]
[i missed you] => [te extrane]
[i need a cab] => [necesito un taxi]


In [6]:

input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (25000,)
Length of target_texts: (25000,)


In [7]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 20
max length of target sentences: 68


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (25000, 20)
shape of input_token_index: 27
shape of decoder_input_seq: (25000, 68)
shape of target_token_index: 29


In [9]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


In [10]:
target_texts[100]


'\tsed buenas\n'

In [11]:
decoder_input_seq[100, :]


array([ 6,  5,  2, 15,  1, 18, 13,  2,  8,  3,  5,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [12]:
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(25000, 20, 28)
(25000, 68, 30)


In [13]:

from keras.layers import Input, LSTM
from keras.models import Model
from keras.layers import Bidirectional, Concatenate
from keras import layers

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
#encoder_lstm = LSTM(latent_dim, return_state=True, 
#                    dropout=0.5, name='encoder_lstm')
#_, state_h, state_c = encoder_lstm(encoder_inputs)


encoder_bilstm = Bidirectional(layers.LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.png'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 28)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 583680      encoder_inputs[0][0]             
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 512)          0           bidirectional_1[0][2]      

In [0]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.layers import Bidirectional, Concatenate
from keras import layers

# inputs of the decoder network
decoder_input_h = Input(shape=(512,), name='decoder_input_h')
decoder_input_c = Input(shape=(512,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(512, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])
    


#state_h = Concatenate()([forward_h, backward_h])
#state_c = Concatenate()([forward_c, backward_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

In [16]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.png'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input_x (InputLayer)    (None, None, 30)     0                                            
__________________________________________________________________________________________________
decoder_input_h (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_input_c (InputLayer)    (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 decoder_input_h[0][0]      

In [0]:

# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [18]:
print (encoder_final_states)


[<tf.Tensor 'encoder/concatenate_1/concat:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'encoder/concatenate_2/concat:0' shape=(?, 512) dtype=float32>]


In [19]:
print(state_h)
print(decoder_input_h)

Tensor("decoder_lstm/while/Exit_2:0", shape=(?, 512), dtype=float32)
Tensor("decoder_input_h:0", shape=(?, 512), dtype=float32)


In [20]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.png'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_x (InputLayer)    (None, None, 28)     0                                            
__________________________________________________________________________________________________
decoder_input_x (InputLayer)    (None, None, 30)     0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 512), (None, 583680      encoder_input_x[0][0]            
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 512),  1112064     decoder_input_x[0][0]            
                                                                 encoder[1][0]       

In [21]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(25000, 20, 28)
shape of decoder_input_data(25000, 68, 30)
shape of decoder_target_data(25000, 68, 30)


In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=20, validation_split=0.2)

model.save('seq2seq.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 20000 samples, validate on 5000 samples
Epoch 1/20





20000/20000 [==============================] - 397s 20ms/step - loss: 0.7931 - val_loss: 0.7076
Epoch 2/20
20000/20000 [==============================] - 395s 20ms/step - loss: 0.5836 - val_loss: 0.6273
Epoch 3/20
20000/20000 [==============================] - 400s 20ms/step - loss: 0.5214 - val_loss: 0.5803
Epoch 4/20
20000/20000 [==============================] - 402s 20ms/step - loss: 0.4782 - val_loss: 0.5419
Epoch 5/20
20000/20000 [==============================] - 409s 20ms/step - loss: 0.4467 - val_loss: 0.5175
Epoch 6/20
20000/20000 [==============================] - 409s 20ms/step - loss: 0.4219 - val_loss: 0.4904
Epoch 7/20
20000/20000 [==============================] - 409s 20ms/step - loss: 0.3989 - val_loss: 0.4757
Epoch 8/20
20000/20000 [==============================] - 402s 20ms/step - loss: 0.3794 - val_loss

In [0]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [29]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

-
English:        is he right
Spanish (true):  esta el bien
Spanish (pred):  es verdad
-
English:        is he right
Spanish (true):  se encuentra bien
Spanish (pred):  es verdad
-
English:        is he right
Spanish (true):  ha acertado
Spanish (pred):  es verdad
-
English:        is he right
Spanish (true):  esta en lo cierto
Spanish (pred):  es verdad
-
English:        is he right
Spanish (true):  es lo que el dice
Spanish (pred):  es verdad
-
English:        is it clean
Spanish (true):  esta limpio
Spanish (pred):  es demasiado
-
English:        is it clean
Spanish (true):  esta limpia
Spanish (pred):  es demasiado
-
English:        is it there
Spanish (true):  esta ahi
Spanish (pred):  es verdad
-
English:        is it to go
Spanish (true):  para llevar
Spanish (pred):  es demasiado trabajo
-
English:        is it windy
Spanish (true):  esta ventoso
Spanish (pred):  es demasiado calor
-
English:        is it yours
Spanish (true):  es suyo
Spanish (pred):  es tu dermano
-
English: 

In [25]:
input_sentence = 'why is that'


#Tokenization and One hot encoding
input_sequence1 = numpy.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for t, char in enumerate(input_sentence):
  input_sequence1[0, t, input_token_index[char]] = 1


translated_sentence = decode_sequence(input_sequence1)

print('source sentence is: ' + input_sentence)

print('translated sentence is: ' + translated_sentence)

source sentence is: why is that
translated sentence is: por que es ese



In [26]:
from nltk.translate.bleu_score import corpus_bleu

actual , predicted = list(), list()
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    #/'print('-')
    #print('English:       ', input_texts[seq_index])
    #print('Spanish (true): ', target_texts[seq_index][1:-1])
    #print('Spanish (pred): ', decoded_sentence[0:-1])
    actual.append(target_texts[seq_index][1:-1])
    predicted.append(decoded_sentence[0:-1])
    
print('BLEU-1: %f' % corpus_bleu(actual, predicted,weights=(1, 0, 0, 0)))

BLEU-1: 0.453441


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [27]:
print (actual)


['esta el bien', 'se encuentra bien', 'ha acertado', 'esta en lo cierto', 'es lo que el dice', 'esta limpio', 'esta limpia', 'esta ahi', 'para llevar', 'esta ventoso', 'es suyo', 'es vuestro', 'ella es simpatica', 'es ese tom', 'es todo', 'es eso nuevo', 'eres tu', 'es usted', 'esto es nuevo', 'no puede ser']


In [28]:
print (predicted)


['es verdad', 'es verdad', 'es verdad', 'es verdad', 'es verdad', 'es demasiado', 'es demasiado', 'es verdad', 'es demasiado trabajo', 'es demasiado calor', 'es tu dermano', 'es tu dermano', 'esta el correcto', 'es eso de tom', 'es eso es tendo', 'eso es de nuevo', 'es ese tuyo', 'es ese tuyo', 'esto es suyo', 'no puede ser']
